In [1]:
import boto3
from boto3.dynamodb.conditions import Key, Attr
from dotenv import load_dotenv
import os
import time
from botocore.exceptions import NoCredentialsError, PartialCredentialsError


# Load environment variables from a .env file
load_dotenv()
# Retrieve the environment variables
aws_access_key_id = os.getenv('AWS_ACCESS_KEY_ID')
aws_secret_access_key = os.getenv('AWS_SECRET_ACCESS_KEY')
aws_session_token = os.getenv('AWS_SESSION_TOKEN')
region_name = os.getenv('AWS_REGION')

# Initialize a session using Amazon DynamoDB
session = boto3.Session(
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    aws_session_token=aws_session_token,
    region_name=region_name
)

dynamodb = session.resource('dynamodb')
print("CONEXION EXITOSA")



CONEXION EXITOSA


1 -Crear al menos 3 tablas con dos atributos cada una 

In [2]:
# Función para crear tablas
def create_tables():
    try:
        for table_name, pk in [("Usuarios", "UserID"), ("Productos", "ProductoID"), ("Pedidos", "PedidoID")]:
            table = dynamodb.create_table(
                TableName=table_name,
                KeySchema=[{"AttributeName": pk, "KeyType": "HASH"}],
                AttributeDefinitions=[{"AttributeName": pk, "AttributeType": "S"}],
                ProvisionedThroughput={"ReadCapacityUnits": 5, "WriteCapacityUnits": 5}
            )
            table.wait_until_exists()
            print(f"Tabla {table_name} creada correctamente")
    except Exception as e:
        print(f"Error: {e}")

create_tables()

Tabla Usuarios creada correctamente
Tabla Productos creada correctamente
Tabla Pedidos creada correctamente


2 - Crear tres registros encada tabla

In [3]:
def insertar_registros():
    try:
        usuarios = [
            {"UserID": "U1", "Nombre": "Alice", "Edad": 30},
            {"UserID": "U2", "Nombre": "Bob", "Edad": 25},
            {"UserID": "U3", "Nombre": "Charlie", "Edad": 35},
            {"UserID": "U4", "Nombre": "David", "Edad": 40}
        ]
        
        productos = [
            {"ProductoID": "P1", "Nombre": "Laptop", "Precio": 1000},
            {"ProductoID": "P2", "Nombre": "Mouse", "Precio": 20},
            {"ProductoID": "P3", "Nombre": "Teclado", "Precio": 50},
            {"ProductoID": "P4", "Nombre": "Monitor", "Precio": 200}
        ]
        
        pedidos = [
            {"PedidoID": "O1", "Cliente": "Alice", "ProductoID": "P1"},
            {"PedidoID": "O2", "Cliente": "Bob", "ProductoID": "P2"},
            {"PedidoID": "O3", "Cliente": "Charlie", "ProductoID": "P3"},
            {"PedidoID": "O4", "Cliente": "David", "ProductoID": "P4"}
        ]
        
        for usuario in usuarios:
            table = dynamodb.Table("Usuarios")
            table.put_item(Item=usuario)
        
        for producto in productos:
            table = dynamodb.Table("Productos")
            table.put_item(Item=producto)
        
        for pedido in pedidos:
            table = dynamodb.Table("Pedidos")
            table.put_item(Item=pedido)
        
        print("Registros insertados correctamente")
    except Exception as e:
        print(f"Error al insertar registros: {e}")

insertar_registros()

Registros insertados correctamente


3 - Obtener un registro de cada tabla 

In [4]:
def obtener_un_registro():
    try:
        keys = {
            "Usuarios": {"UserID": "U1"},
            "Productos": {"ProductoID": "P1"},
            "Pedidos": {"PedidoID": "O1"}
        }
        
        for table_name, key in keys.items():
            table = dynamodb.Table(table_name)
            response = table.get_item(Key=key)
            item = response.get('Item', {})
            print(f"Registro en la tabla {table_name}: {item}")
    except Exception as e:
        print(f"Error al obtener un registro: {e}")

obtener_un_registro()

Registro en la tabla Usuarios: {'Nombre': 'Alice', 'UserID': 'U1', 'Edad': Decimal('30')}
Registro en la tabla Productos: {'Precio': Decimal('1000'), 'Nombre': 'Laptop', 'ProductoID': 'P1'}
Registro en la tabla Pedidos: {'Cliente': 'Alice', 'PedidoID': 'O1', 'ProductoID': 'P1'}


4 - Actualizar un registro de cada tabla

In [5]:
def actualizar_registros():
    try:
        updates = [
            ("Usuarios", {"UserID": "U1"}, "SET Nombre = :nombre", {":nombre": "Alice Updated"}),
            ("Productos", {"ProductoID": "P1"}, "SET Nombre = :nombre", {":nombre": "Laptop Updated"}),
            ("Pedidos", {"PedidoID": "O1"}, "SET Cliente = :cliente", {":cliente": "Alice Updated"})
        ]
        
        for table_name, key, update_expression, expression_attribute_values in updates:
            table = dynamodb.Table(table_name)
            table.update_item(
                Key=key,
                UpdateExpression=update_expression,
                ExpressionAttributeValues=expression_attribute_values
            )
            print(f"Registro actualizado en la tabla {table_name}: {key}")
    except Exception as e:
        print(f"Error al actualizar registros: {e}")

actualizar_registros()

Registro actualizado en la tabla Usuarios: {'UserID': 'U1'}
Registro actualizado en la tabla Productos: {'ProductoID': 'P1'}
Registro actualizado en la tabla Pedidos: {'PedidoID': 'O1'}


5 - Eliminar un registro de cada tabla 

In [6]:
def eliminar_registros():
    try:
        keys = {
            "Usuarios": {"UserID": "U1"},
            "Productos": {"ProductoID": "P1"},
            "Pedidos": {"PedidoID": "O1"}
        }
        
        for table_name, key in keys.items():
            table = dynamodb.Table(table_name)
            table.delete_item(Key=key)
            print(f"Registro eliminado en la tabla {table_name}: {key}")
    except Exception as e:
        print(f"Error al eliminar registros: {e}")

eliminar_registros()

Registro eliminado en la tabla Usuarios: {'UserID': 'U1'}
Registro eliminado en la tabla Productos: {'ProductoID': 'P1'}
Registro eliminado en la tabla Pedidos: {'PedidoID': 'O1'}


6 - Obtener todos los registros de cada tabla

In [7]:
def obtener_todos_los_registros():
    try:
        table_names = ["Usuarios", "Productos", "Pedidos"]
        
        for table_name in table_names:
            table = dynamodb.Table(table_name)
            response = table.scan()
            items = response.get('Items', [])
            print(f"Registros en la tabla {table_name}: {items}")
    except Exception as e:
        print(f"Error al obtener todos los registros: {e}")

obtener_todos_los_registros()

Registros en la tabla Usuarios: [{'Nombre': 'David', 'UserID': 'U4', 'Edad': Decimal('40')}, {'Nombre': 'Charlie', 'UserID': 'U3', 'Edad': Decimal('35')}, {'Nombre': 'Bob', 'UserID': 'U2', 'Edad': Decimal('25')}]
Registros en la tabla Productos: [{'Precio': Decimal('20'), 'Nombre': 'Mouse', 'ProductoID': 'P2'}, {'Precio': Decimal('50'), 'Nombre': 'Teclado', 'ProductoID': 'P3'}, {'Precio': Decimal('200'), 'Nombre': 'Monitor', 'ProductoID': 'P4'}]
Registros en la tabla Pedidos: [{'Cliente': 'Charlie', 'PedidoID': 'O3', 'ProductoID': 'P3'}, {'Cliente': 'David', 'PedidoID': 'O4', 'ProductoID': 'P4'}, {'Cliente': 'Bob', 'PedidoID': 'O2', 'ProductoID': 'P2'}]


7 - Obtener una conjunto de registros de un filtrado de cada tabla 

In [8]:
def obtener_registros_filtrados():
    try:
        filtros = {
            "Usuarios": Attr("Edad").gt(30),
            "Productos": Attr("Precio").gt(50),
            "Pedidos": Attr("Cliente").eq("Alice Updated")
        }
        
        for table_name, filtro in filtros.items():
            table = dynamodb.Table(table_name)
            response = table.scan(FilterExpression=filtro)
            items = response.get('Items', [])
            print(f"Registros filtrados en la tabla {table_name}: {items}")
    except Exception as e:
        print(f"Error al obtener registros filtrados: {e}")

obtener_registros_filtrados()

Registros filtrados en la tabla Usuarios: [{'Nombre': 'David', 'UserID': 'U4', 'Edad': Decimal('40')}, {'Nombre': 'Charlie', 'UserID': 'U3', 'Edad': Decimal('35')}]
Registros filtrados en la tabla Productos: [{'Precio': Decimal('200'), 'Nombre': 'Monitor', 'ProductoID': 'P4'}]
Registros filtrados en la tabla Pedidos: []


8 - Realizar una eliminación condicional de cada tabla 

In [9]:
def eliminacion_condicional():
    try:
        condiciones = [
            ("Usuarios", {"UserID": "U2"}, Attr("Edad").lt(30)),
            ("Productos", {"ProductoID": "P2"}, Attr("Precio").lt(50)),
            ("Pedidos", {"PedidoID": "O2"}, Attr("Cliente").eq("Bob"))
        ]
        
        for table_name, key, condition in condiciones:
            table = dynamodb.Table(table_name)
            table.delete_item(
                Key=key,
                ConditionExpression=condition
            )
            print(f"Registro eliminado condicionalmente en la tabla {table_name}: {key}")
    except Exception as e:
        print(f"Error al eliminar registros condicionalmente: {e}")

eliminacion_condicional()

Registro eliminado condicionalmente en la tabla Usuarios: {'UserID': 'U2'}
Registro eliminado condicionalmente en la tabla Productos: {'ProductoID': 'P2'}
Registro eliminado condicionalmente en la tabla Pedidos: {'PedidoID': 'O2'}


9 - Obtener un conjunto de datos a través de varios filtros aplicado en cada tabla

In [10]:
def obtener_registros_varios_filtros():
    try:
        filtros = {
            "Usuarios": Attr("Edad").between(25, 35) & Attr("Nombre").begins_with("A"),
            "Productos": Attr("Precio").between(20, 200) & Attr("Nombre").contains("o"),
            "Pedidos": Attr("Cliente").begins_with("A") & Attr("ProductoID").begins_with("P")
        }
        
        for table_name, filtro in filtros.items():
            table = dynamodb.Table(table_name)
            response = table.scan(FilterExpression=filtro)
            items = response.get('Items', [])
            print(f"Registros filtrados en la tabla {table_name} con varios filtros: {items}")
    except Exception as e:
        print(f"Error al obtener registros con varios filtros: {e}")

obtener_registros_varios_filtros()

Registros filtrados en la tabla Usuarios con varios filtros: []
Registros filtrados en la tabla Productos con varios filtros: [{'Precio': Decimal('50'), 'Nombre': 'Teclado', 'ProductoID': 'P3'}, {'Precio': Decimal('200'), 'Nombre': 'Monitor', 'ProductoID': 'P4'}]
Registros filtrados en la tabla Pedidos con varios filtros: []


10 - Utilizar PartiQL statement en cada tabla

In [11]:
def utilizar_partiql():
    try:
        statements = [
            "SELECT * FROM Usuarios WHERE Edad > 30",
            "SELECT * FROM Productos WHERE Precio > 50",
            "SELECT * FROM Pedidos WHERE Cliente = 'Alice Updated'"
        ]
        
        for statement in statements:
            response = dynamodb.meta.client.execute_statement(Statement=statement)
            items = response.get('Items', [])
            print(f"Resultados para la consulta '{statement}': {items}")
    except Exception as e:
        print(f"Error al ejecutar PartiQL: {e}")

utilizar_partiql()

Resultados para la consulta 'SELECT * FROM Usuarios WHERE Edad > 30': [{'Nombre': 'David', 'UserID': 'U4', 'Edad': Decimal('40')}, {'Nombre': 'Charlie', 'UserID': 'U3', 'Edad': Decimal('35')}]
Resultados para la consulta 'SELECT * FROM Productos WHERE Precio > 50': [{'Precio': Decimal('200'), 'Nombre': 'Monitor', 'ProductoID': 'P4'}]
Resultados para la consulta 'SELECT * FROM Pedidos WHERE Cliente = 'Alice Updated'': []


11 - Crear un backup de todas las tablas 

In [12]:
def crear_backup():
    try:
        table_names = ["Usuarios", "Productos", "Pedidos"]
        
        for table_name in table_names:
            backup_name = f"{table_name}_backup_{int(time.time())}"
            response = dynamodb.meta.client.create_backup(
                TableName=table_name,
                BackupName=backup_name
            )
            print(f"Backup creado para la tabla {table_name}: {response['BackupDetails']['BackupArn']}")
    except Exception as e:
        print(f"Error al crear backup: {e}")

crear_backup()

Backup creado para la tabla Usuarios: arn:aws:dynamodb:us-east-1:284511614577:table/Usuarios/backup/01738784906415-c2b9f014
Backup creado para la tabla Productos: arn:aws:dynamodb:us-east-1:284511614577:table/Productos/backup/01738784906602-249ce01e
Backup creado para la tabla Pedidos: arn:aws:dynamodb:us-east-1:284511614577:table/Pedidos/backup/01738784906764-e0ee6e6f


In [13]:
create_tables()
insertar_registros()
obtener_un_registro()
actualizar_registros()
eliminar_registros()
obtener_todos_los_registros()
obtener_registros_filtrados()
eliminacion_condicional()
obtener_registros_varios_filtros()
utilizar_partiql()
crear_backup()

Error: An error occurred (ResourceInUseException) when calling the CreateTable operation: Table already exists: Usuarios
Registros insertados correctamente
Registro en la tabla Usuarios: {'Nombre': 'Alice', 'UserID': 'U1', 'Edad': Decimal('30')}
Registro en la tabla Productos: {'Precio': Decimal('1000'), 'Nombre': 'Laptop', 'ProductoID': 'P1'}
Registro en la tabla Pedidos: {'Cliente': 'Alice', 'PedidoID': 'O1', 'ProductoID': 'P1'}
Registro actualizado en la tabla Usuarios: {'UserID': 'U1'}
Registro actualizado en la tabla Productos: {'ProductoID': 'P1'}
Registro actualizado en la tabla Pedidos: {'PedidoID': 'O1'}
Registro eliminado en la tabla Usuarios: {'UserID': 'U1'}
Registro eliminado en la tabla Productos: {'ProductoID': 'P1'}
Registro eliminado en la tabla Pedidos: {'PedidoID': 'O1'}
Registros en la tabla Usuarios: [{'Nombre': 'David', 'UserID': 'U4', 'Edad': Decimal('40')}, {'Nombre': 'Charlie', 'UserID': 'U3', 'Edad': Decimal('35')}, {'Nombre': 'Bob', 'UserID': 'U2', 'Edad': De